In [1]:
%load_ext autoreload
%autoreload 2
import glob 
import json 

evaluation_files = glob.glob("../evaluations/emotion_*.json")

evaluations = []

for file in evaluation_files:
    print(file)
    with open(file) as f:
        evaluation = json.load(f)
        evaluation["file"] = file.split("/")[-1]
        evaluations.append(evaluation)

../evaluations/emotion_bertweet_base.json
../evaluations/emotion_beto.json
../evaluations/emotion_roberta.json
../evaluations/emotion_bert_base.json


In [8]:
import pandas as pd 

df = pd.DataFrame([
    {**evaluation, **evaluation["metrics"]} for evaluation in evaluations
]).set_index("file")

df.drop(labels=["predictions", "labels", "metrics"], inplace=True, axis=1)

df.sort_values(["lang", "test_macro_f1"])

,model,lang,test_loss,test_others_f1,test_others_precision,test_others_recall,test_joy_f1,test_joy_precision,test_joy_recall,test_sadness_f1,...,test_disgust_recall,test_fear_f1,test_fear_precision,test_fear_recall,test_macro_f1,test_macro_precision,test_macro_recall,test_acc,test_runtime,test_samples_per_second
file,,,,,,,,,,,,,,,,,,,,,
emotion_bert_base.json,models/bert-base-emotion-analysis/,en,1.448668,0.660011,0.609007,0.720339,0.680121,0.700624,0.660784,0.565445,...,0.371728,0.253968,0.320000,0.210526,0.441858,0.486801,0.415784,0.601862,10.7134,170.440
emotion_bertweet_base.json,models/bertweet-base-emotion-analysis/,en,1.188743,0.663127,0.646720,0.680387,0.711198,0.712598,0.709804,0.582960,...,0.486911,0.166667,0.400000,0.105263,0.459780,0.497312,0.452507,0.618291,9.6992,188.262
emotion_roberta.json,models/roberta-base-emotion-analysis/,en,1.182252,0.650000,0.654791,0.645278,0.722800,0.698355,0.749020,0.564315,...,0.549738,0.273973,0.285714,0.263158,0.473204,0.502806,0.471303,0.620482,11.0043,165.934
emotion_beto.json,models/beto-emotion-analysis/,es,1.216769,0.762615,0.725191,0.804111,0.639087,0.660767,0.618785,0.756892,...,0.000000,0.511628,0.440000,0.611111,0.500856,0.507610,0.505274,0.688730,12.2470,136.932
